<a href="https://colab.research.google.com/github/peace-and-harmony/image-matting/blob/main/notebooks/modnet_demo_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/Pronti-Beam/demos-experiments/blob/master/notebooks/u_2net_demo_benchmark_edition.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/Pronti-Beam/blob/master/demos-experiments/notebooks/u_2net_demo_benchmark_edition.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td><td>
  <a target="_blank"  href="https://www.linkedin.com/company/pronti-ai/?originalSubdomain=ca">
    <img width=128px src="https://miro.medium.com/max/3402/1*gPSJe7WqcC61cGyB0lxalQ.png" /></a>
</td><td>
  <a target="_blank"  href="http://beamdata.ca/">
    <img width=128px src="http://beamdata.ca/wp-content/uploads/2018/10/beamdata_web_logo.png" /></a>
</td></table>


# MODNet benchmark demo

In this demo, the differece between Pytorch and ONNX runtime is compared.

**Note** Use Runtime: CPU as base of the benchmark

 ---

The model type and inference runtime are listed in the Table:

Model name  | Inference type | Runtime per image (ms)
-------------------|------------------|-----------------
checkpoint.pth       | Pytorch  | 1302.6847
checkpoint.onnx       | ONNX runtime    | 848.1648 
checkpoint-simplified.onnx       | ONNX runtime     | 824.7533

---

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Load the test dataset for inference runtime benchmark

In [ ]:
%cd /content
!ls /content/drive/MyDrive/Cropper
!cp /content/drive/MyDrive/Cropper/cropper_validation.zip /content
!unzip cropper_validation.zip

In [3]:
%cd /content/valid_validation
!mkdir test_run
!cp -r image test_run

/content/valid_validation


In [47]:
import glob
len(glob.glob('/content/valid_validation/mask/*.jpeg'))

1127

#### Download the pre-trained MODNet model

In [4]:
%cd /content
!git clone https://github.com/ZHKKKe/MODNet.git

/content
Cloning into 'MODNet'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 249 (delta 21), reused 24 (delta 9), pack-reused 206
Receiving objects: 100% (249/249), 60.76 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [7]:
!mkdir -p /content/MODNet/test_data/test_human_images_results/
%ls /content/MODNet/test_data/

test_human_images/  test_human_images_results/


# Generate .onnx
- Optimisation

ValueError: Unsupported ONNX opset version: 13

  - opset_version=12



The next step should we decide to continue and pursuse this avenue would be to investigate whether or need this particular graph is rejected by TesnorRT's onnnx parser, or alternatively coreml etc should you want to deploy directly on the edge.

#### install the requirements for converting MODNet to .onnx

In [ ]:
%cd /content/MODNet
!pip install -r onnx/requirements.txt

In [8]:
%ls /content/drive/MyDrive/Beamdata-Pronti/modelsv2/

_bcev2_itr_101297_tar_29.000000.pth  _bcev2_itr_41916_tar_12.000000.pth
_bcev2_itr_104790_tar_30.000000.pth  _bcev2_itr_45409_tar_13.000000.pth
_bcev2_itr_10479_tar_3.000000.pth    _bcev2_itr_48902_tar_14.000000.pth
_bcev2_itr_108283_tar_31.000000.pth  _bcev2_itr_52395_tar_15.000000.pth
_bcev2_itr_111776_tar_32.000000.pth  _bcev2_itr_55888_tar_16.000000.pth
_bcev2_itr_115269_tar_33.000000.pth  _bcev2_itr_59381_tar_17.000000.pth
_bcev2_itr_118762_tar_34.000000.pth  _bcev2_itr_62874_tar_18.000000.pth
_bcev2_itr_122255_tar_35.000000.pth  _bcev2_itr_66367_tar_19.000000.pth
_bcev2_itr_125748_tar_36.000000.pth  _bcev2_itr_69860_tar_20.000000.pth
_bcev2_itr_129241_tar_37.000000.pth  _bcev2_itr_6986_tar_2.000000.pth
_bcev2_itr_132734_tar_38.000000.pth  _bcev2_itr_73353_tar_21.000000.pth
_bcev2_itr_136472_tar_39.000000.pth  _bcev2_itr_76846_tar_22.000000.pth
_bcev2_itr_13972_tar_4.000000.pth    _bcev2_itr_80339_tar_23.000000.pth
_bcev2_itr_140210_tar_40.000000.pth  _bcev2_itr_83832_tar_24.00000

In [1]:
!cp /content/drive/MyDrive/Beamdata-Pronti/modelsv2/_bcev2_itr_158900_tar_45.000000.pth /content/MODNet/pretrained/

### Export to cpu-based .onnx

In [19]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet_photographic_portrait_matting.ckpt
modnet-cpu.onnx                      README.md


In [14]:
%pwd

'/content'

In [10]:
%ls /content/MODNet/

demo/  doc/  onnx/  pretrained/  README.md  src/  test_data/  torchscript/


/content/MODNet


In [22]:
%cd /content/MODNet


import os
import torch
import torch.nn as nn
from torch.autograd import Variable

from MODNet.src.models.modnet import MODNet
from MODNet.onnx import modnet_onnx


# general input
input_name = '/content/MODNet/pretrained/_bcev2_itr_158900_tar_45.000000.pth'
# check input arguments
if not os.path.exists(input_name):
    print('Cannot find checkpoint path: {0}'.format(ckpt_path))
    exit()

# define model & load checkpoint
modnet = modnet_onnx.MODNet(backbone_pretrained=False)

# prepare dummy_input
batch_size = 1
height = 512
width = 512

# dummy_input: input tensor x. The values in this can be random as long as it is the right type and size
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using gpu!')
  dummy_input = Variable(torch.randn(batch_size, 3, height, width)).cuda()
  modnet = nn.DataParallel(modnet).cuda()

else:
  device = torch.device('cpu')
  print('using cpu')
  dummy_input = Variable(torch.randn(batch_size, 3, height, width))
  modnet = nn.DataParallel(modnet)

state_dict = torch.load(input_name, map_location=device)
modnet.load_state_dict(state_dict['state_dict'])
modnet.eval() # set the model to inference mode

if torch.cuda.is_available():
  output_name = '/content/MODNet/pretrained/modnet-gpu.onnx'
else:
  output_name = '/content/MODNet/pretrained/modnet-45-cpu.onnx'


# export to onnx model
torch.onnx.export(
    modnet.module, dummy_input, output_name, export_params = True, 
    input_names = ['input'], output_names = ['output'], 
    dynamic_axes = {'input': {0:'batch_size', 2:'height', 3:'width'}, 'output': {0: 'batch_size', 2: 'height', 3: 'width'}}, opset_version=12)

%ls /content/MODNet/pretrained/

/content/MODNet
using cpu


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


_bcev2_itr_158900_tar_45.000000.pth  modnet_photographic_portrait_matting.ckpt
modnet-45-cpu.onnx                   README.md
modnet-cpu.onnx


.onnx is a binary protobuf file which contains both the network structure and parameters of the model

Save modnet-cpu.onnx to drive for further onnx simplifier later

In [70]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet-gpu-simplified.onnx
modnet-45-gpu.onnx                   modnet_photographic_portrait_matting.ckpt
modnet-45-gpu-simplified.onnx        README.md
modnet-gpu.onnx


In [72]:
!cp /content/MODNet/pretrained/modnet-45-gpu-simplified.onnx /content/drive/MyDrive/Beamdata-Pronti/models/

In [73]:
%ls /content/drive/MyDrive/Beamdata-Pronti/models/

_bce_itr_101297_tar_29.000000.pth  _bce_itr_69860_tar_20.000000.pth
_bce_itr_104790_tar_30.000000.pth  _bce_itr_6986_tar_2.000000.pth
_bce_itr_10479_tar_3.000000.pth    _bce_itr_73353_tar_21.000000.pth
_bce_itr_108283_tar_31.000000.pth  _bce_itr_76846_tar_22.000000.pth
_bce_itr_111776_tar_32.000000.pth  _bce_itr_80339_tar_23.000000.pth
_bce_itr_115269_tar_33.000000.pth  _bce_itr_83832_tar_24.000000.pth
_bce_itr_13972_tar_4.000000.pth    _bce_itr_87325_tar_25.000000.pth
_bce_itr_17465_tar_5.000000.pth    _bce_itr_90818_tar_26.000000.pth
_bce_itr_20958_tar_6.000000.pth    _bce_itr_94311_tar_27.000000.pth
_bce_itr_24451_tar_7.000000.pth    _bce_itr_97804_tar_28.000000.pth
_bce_itr_27944_tar_8.000000.pth    logs/
_bce_itr_31437_tar_9.000000.pth    mobilenetv2_human_seg.ckpt
_bce_itr_34930_tar_10.000000.pth   modnet-45-gpu.onnx
_bce_itr_3493_tar_1.000000.pth     modnet-45-gpu-simplified.onnx
_bce_itr_38423_tar_11.000000.pth   modnet-cpu.onnx
_bce_itr_41916_tar_12.000000.pth   modnet-cpu-onn

### Output of PyTorch and ONNX match test
  - trace-based means that it operates by executing your model once
  - the output of PyTorch and ONNX Runtime runs match numerically with the given precision (rtol=1e-03 and atol=1e-05)
  - onnx exporter is correct

In [30]:
import onnxruntime
import numpy as np


# Input to the model
x = torch.randn(1, 3, 512, 512, requires_grad=True)
torch_out = modnet(x)

ort_session = onnxruntime.InferenceSession("/content/MODNet/pretrained/modnet-45-cpu.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Exported model has been tested with ONNXRuntime, and the result looks good!


#ONNX Runtime inference

Use converted checkpoint.onnx for inference

In [26]:
!pip install onnxruntime
!pip install onnx

In [52]:
import glob
import os

%cd /content/MODNet/
img_name_list = glob.glob('/content/valid_validation/image/' + os.sep + '*')

/content/MODNet


In [58]:
!mkdir -p /content/valid_validation/image/test_run
!cp /content/valid_validation/image/* /content/valid_validation/image/test_run

cp: -r not specified; omitting directory '/content/valid_validation/image/test_run'


In [61]:
import torchvision
from torchvision import transforms

val_data = "/content/valid_validation/image"

TRANSFORM_IMG = transforms.Compose([
     transforms.Resize((512, 512)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

val = torchvision.datasets.ImageFolder(val_data, transform=TRANSFORM_IMG)

In [62]:
# ONNX Runtime Inference

from torch.utils.data import Dataset, DataLoader

import onnxruntime as rt  
import time
from tqdm.notebook import tqdm

n_runs = 50

sess_options = rt.SessionOptions()

sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

session = rt.InferenceSession('/content/MODNet/pretrained/modnet-45-cpu.onnx', sess_options=sess_options)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dynamic = False
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=2)
        batch = next(iter(dataloader))
        batch = tuple(t.to('cpu') for t in batch)

        start = time.time()
        pred = session.run([output_name], {input_name: batch[0].numpy()})[0]
        end = time.time()
        runtimes.append((end-start)*1000)

    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]

inference cost for batch_size 1: 848.1648 ms
overall inference execution cost: 56.2741 seconds


# Pytorch comparison
Use pretrained checkpoint.pth via Pytorch for inference




In [67]:
import time
from tqdm.notebook import tqdm

dynamic = False

# load MODNet and pretrained checkpoint
input_name = '/content/MODNet/pretrained/_bcev2_itr_158900_tar_45.000000.pth'
# check input arguments
if not os.path.exists(input_name):
    print('Cannot find checkpoint path: {0}'.format(ckpt_path))
    exit()

# define model & load checkpoint
modnet = modnet_onnx.MODNet(backbone_pretrained=False)

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using gpu!')
  modnet = nn.DataParallel(modnet).cuda()

else:
  device = torch.device('cpu')
  print('using cpu')
  modnet = nn.DataParallel(modnet)

state_dict = torch.load(input_name, map_location=device)
modnet.load_state_dict(state_dict['state_dict'])
modnet.eval() # set the model to inference mode

if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

n_runs = 50

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=2)
        batch = next(iter(dataloader))
        batch = tuple(t.to('cpu') for t in batch)

        start = time.time()
        matte = modnet(batch[0])
        end = time.time()
        runtimes.append((end-start)*1000)

    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)}ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

using cpu


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


inference cost for batch_size 1: 1302.6847ms
overall inference execution cost: 83.5205 seconds


## ONNX Simplifier
replaces the redundant operators with their constant outputs to simplify onnx model.

In [ ]:
%cd /content/
#Otherwise we get a random pad node error
!pip3 install -U pip && pip3 install onnx-simplifier

In [40]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet_photographic_portrait_matting.ckpt
modnet-45-cpu.onnx                   README.md
modnet-cpu.onnx


In [41]:
!python3 -m onnxsim /content/MODNet/pretrained/modnet-45-cpu.onnx /content/MODNet/pretrained/modnet-45-cpu-simplified.onnx --input-shape 1,3,512,512

Simplifying...
Note: The input shape of the simplified model will be overwritten by the value of '--input-shape' argument. Pass '--dynamic-input-shape' if it is not what you want. Run 'python3 -m onnxsim -h' for details.
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


In [63]:
!du /content/MODNet/pretrained/modnet-gpu.onnx

25284	/content/MODNet/pretrained/modnet-gpu.onnx


In [64]:
!du /content/MODNet/pretrained/modnet-gpu-simplified.onnx

25276	/content/MODNet/pretrained/modnet-gpu-simplified.onnx


In [68]:
# ONNX Runtime Inference

from torch.utils.data import Dataset, DataLoader

import onnxruntime as rt  
import time
from tqdm.notebook import tqdm

n_runs = 50

sess_options = rt.SessionOptions()

sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# To enable model serialization and store the optimized graph to desired location.
#sess_options.optimized_model_filepath = '/content/esnet/optimized/esnet.onnx'
session = rt.InferenceSession('/content/MODNet/pretrained/modnet-45-cpu-simplified.onnx', sess_options=sess_options)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dynamic = False
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=2)
        batch = next(iter(dataloader))
        batch = tuple(t.to('cpu') for t in batch)

        start = time.time()
        pred = session.run([output_name], {input_name: batch[0].numpy()})[0]
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shapredictions, bitmask = torch.max(pred_torch, 1)pe)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]

inference cost for batch_size 1: 824.7533 ms
overall inference execution cost: 58.2651 seconds


## Model Visualization via Netron

In [59]:
!pip install -q netron

     |████████████████████████████████| 1.3 MB 8.3 MB/s 


In [57]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet-gpu-simplified.onnx
modnet-45-gpu.onnx                   modnet_photographic_portrait_matting.ckpt
modnet-gpu.onnx                      README.md


In [61]:
import netron
import portpicker
from google.colab import output

port = portpicker.pick_unused_port()

# Read the model file and start the netron browser.
with output.temporary():
  netron.start('/content/MODNet/pretrained/modnet-gpu.onnx', address=port, browse=True)

output.serve_kernel_port_as_iframe(port, height='800')

<IPython.core.display.Javascript object>

In [62]:
import netron
import portpicker
from google.colab import output

port = portpicker.pick_unused_port()

# Read the model file and start the netron browser.
with output.temporary():
  netron.start('/content/MODNet/pretrained/modnet-gpu-simplified.onnx', address=port, browse=True)

output.serve_kernel_port_as_iframe(port, height='800')

<IPython.core.display.Javascript object>